In [2]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", 'spanish')

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label          source
0  estoy hasta el ojete de que me digáis que teng...  negative  intertass_2017
1  @user Por?  Tenía pensado verla después de la ...   neutral  intertass_2017
2                           Esto de estar feliz mola  positive  intertass_2017
3                             Ya no es tan divertido  negative  intertass_2017
4  @user te recuerdo que soy una persona que tien...   neutral  intertass_2017
----------------------------------------------------------------------------------------------------


Generating validation split:   0%|          | 0/324 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label          source
0  @user La prensa cada vez más desacreditada  De...  negative  intertass_2017
1  Uds locas por sun and moon me dieron ganas de ...   neutral  intertass_2017
2  @user Buenos Vídeos.Se que llegadas a los 1.00...  positive  intertass_2017
3  El amanecer va a ser bien feo por el recuento ...  negative  intertass_2017
4  Hoy estaré en @user en zona de Humor ups! De A...   neutral  intertass_2017
----------------------------------------------------------------------------------------------------


Generating test split:   0%|          | 0/870 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label          source
0  @user jajajaja dale, hacete la boluda vos jaja...  negative  intertass_2017
1  cada vez que cito un tweet se va la ubicación ...   neutral  intertass_2017
2  @user MAAAAE RAJADO! Pero lo bueno es q uno se...  positive  intertass_2017
3  Bueno hoy fui a almorzar a Nanay con otras 3 d...  negative  intertass_2017
4  Necesito seguir a mas cuentas camren shippers ...   neutral  intertass_2017
----------------------------------------------------------------------------------------------------
Dataset multilingual-sentiments downloaded and prepared to /home/stare/.cache/huggingface/datasets/tyqiangz___multilingual-sentiments/spanish/1.0.0/b7cdd8874d82679e59432edf79e074f595c4ad26d2e562eba4fb55f361691b07. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 870
    })
})

### Clean Dataset

In [60]:
def cleanData(data):
    import re
    data['text'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t!¿¡üÜáéíóúÁÉÍÓÚñÑ])|(\w+:\/\/\S+)"," ", data['text']).split())
    return data

In [61]:
updated_dataset = dataset.map(cleanData)

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

In [62]:
for i in updated_dataset['train']:
    print(i['text'])

estoy hasta el ojete de que me digáis que tengo cara de mala leche
Por Tenía pensado verla después de la segunda de Daredevil
Esto de estar feliz mola
Ya no es tan divertido
te recuerdo que soy una persona que tiene criterio equivocado pero lo tengo
con una pequeña donación hará felices a miles de chicas que no tienen asociacionmariloli
He probado una nueva espuma para el pelo y sí que lo deja más rizado pero se queda como efecto gomina y no me gusta
Ojalá pudiera darte el abrazo en vivo o al menos acompañarte hoy Siento mucho lo de tu primo
aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español Os llevareis bien
ya somos dos que triste
JUAS ahora dicen que la paja se la van a hacer con Tracer y su novia Porque siempre se puede caer más bajo aún ¿no
2AñosDeLegiónHolk Llevo como unos 8 meses Talvez menos o mas no sé Me enseñaron y demostraron que LH es algo magico
severus snape pobresito nunca le odié me parecía muy estricto e injusto pero sabía que algo pasaba
es ch

### Train

In [65]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [135]:
texts = updated_dataset["train"]["text"]
sentiments = updated_dataset["train"]["label"]

texts_test = updated_dataset["test"]["text"]
sentiments_test = updated_dataset["test"]["label"]

texts_val = updated_dataset["validation"]["text"]
sentiments_val = updated_dataset["validation"]["label"]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)

X_train = vectorizer.fit_transform(texts).toarray()
y_train = sentiments

X_test = vectorizer.fit_transform(texts_test).toarray()
y_test = sentiments_test

X_val = vectorizer.fit_transform(texts_val).toarray()
y_val = sentiments_val

In [138]:
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) >= max_length:
            padded_sequence = sequence[:max_length]
        else:
            padded_sequence = sequence + [0] * (max_length - len(sequence))
        padded_sequences.append(padded_sequence)
    return np.array(padded_sequences)


In [136]:
# Initialize the XGBoost model

model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        eta=0.01,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        random_state=42
)

# Train the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eta=0.01, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_class=3, num_parallel_tree=None, ...)

In [139]:
X_padded = pad_sequences(X_test.tolist(), 5000)

In [145]:
X_train.shape, X_padded.shape

((1839, 5000), (870, 5000))

In [146]:
# Make predictions on the test data
y_pred = model.predict(X_padded)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3413793103448276


In [147]:
import pandas as pd
import numpy as np
dftest = pd.DataFrame(np.array(y_test).reshape(-1, 1))
dftarget = pd.DataFrame(np.array(y_pred).reshape(-1, 1))
pd.concat([dftest,dftarget], axis=1)

,0,0
0,2,1
1,1,1
2,0,1
3,2,1
4,1,1
...,...,...
865,1,1
866,0,1
867,2,1
868,1,0


In [123]:
model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        eta=0.01,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        random_state=42
)

# Train the model on the training data
model.fit(X, sentiments)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eta=0.01, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_class=3, num_parallel_tree=None, ...)

In [ ]:
validation = updated_dataset["validation"]["text"]
sentiments = updated_dataset["train"]["label"]
Xv = vectorizer.fit_transform(validation).toarray()

X_train, X_test, y_train, y_test = train_test_split(Xv, sentiments, test_size=0.2, random_state=42)